In [1]:
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
from keras import backend as K
from keras_vggface import utils
from keras.models import load_model

#custom parameters
hidden_dim = 512
image_size = 224
train_dir = "dataset_fgnet"
validation_dir = "dataset_valid_deep"
classes = 10
EPOCHS = 50

custom_vgg_model = load_model('VGGFace50')
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from keras.preprocessing.image import ImageDataGenerator 

train_datagen = ImageDataGenerator(
      preprocessing_function=utils.preprocess_input,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(preprocessing_function=utils.preprocess_input)
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 30
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 554 images belonging to 10 classes.
Found 1363 images belonging to 10 classes.


In [3]:
EPOCHS = 10
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
custom_vgg_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = custom_vgg_model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/10
28/27 [==============================] - 31s 1s/step - loss: 1.9997 - acc: 0.4772 - val_loss: 2.4137 - val_acc: 0.2575
Epoch 2/10
28/27 [==============================] - 22s 779ms/step - loss: 1.0749 - acc: 0.5686 - val_loss: 1.7012 - val_acc: 0.3844
Epoch 3/10
28/27 [==============================] - 21s 768ms/step - loss: 0.8524 - acc: 0.6572 - val_loss: 1.7444 - val_acc: 0.4079
Epoch 4/10
28/27 [==============================] - 22s 787ms/step - loss: 0.7882 - acc: 0.6605 - val_loss: 1.6391 - val_acc: 0.4175
Epoch 5/10
28/27 [==============================] - 22s 791ms/step - loss: 0.7258 - acc: 0.7026 - val_loss: 2.0082 - val_acc: 0.3793
Epoch 6/10
28/27 [==============================] - 22s 787ms/step - loss: 0.6629 - acc: 0.7269 - val_loss: 2.2660 - val_acc: 0.3617
Epoch 7/10
28/27 [==============================] - 22s 773ms/step - loss: 0.5919 - acc: 0.7538 - val_loss: 2.1345 - val_acc: 0.3888
Epoch 8/10
28/27 [==============================] - 22s 799ms/step - los

In [4]:
import matplotlib.pyplot as plt
import numpy as np

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("VGGFace50FGNET")

In [6]:
import csv
import os
import cv2
from keras.preprocessing.image import img_to_array
custom_vgg_model = load_model('VGGFace50')

def readFaces(dir,mydict):
    faces = []
    ages = []
    directory = os.fsencode(dir)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".png") or filename.endswith(".jpg"): 
            path = ".\\" + dir + "\\" + filename
            img = cv2.imread(path)
            img = cv2.resize(img, (224, 224))
            img = img.astype("float16")
            img = utils.preprocess_input(x=img,version=1)
            img = img_to_array(img)
            img = np.expand_dims(img, axis=0)
            faces.append(img)
            ages.append(mydict.get(filename))
    return np.asarray(faces),ages

with open('ground_truth/test_gt.csv', 'r') as f:
    reader = csv.reader(f)
    mydict = dict((rows[0],rows[1]) for rows in reader)

x_test,y_test = readFaces("test_deep",mydict)

def arrangeLabels(Y,bins,labels):
    returnLabels = []
    for label in Y:
        for index in range(len(bins)):
            if float(label) > bins[index]:
                continue;
            else:
                returnLabels.append(labels[index-1])
                break;
                
    return returnLabels

bins = [0.0, 6.0, 13.0, 20.0, 27.0, 34.0, 41.0, 48.0, 55.0, 62.0,np.inf]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

y_test = np.array(arrangeLabels(y_test,bins,labels))

hit = 0
total = len(x_test)
for index in range(total):
    # classify the input image
    print("[INFO] classifying image...")
    proba = custom_vgg_model.predict(x_test[index])[0]
    idx = np.argmax(proba)
    print("Predicted "+str(idx)," Actual "+str(y_test[index]))
    hit += idx==y_test[index]

print(hit/total)


[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 7  Actual 9
[INFO] classifying image...
Predicted 5  Actual 8
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 5  Actual 3
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 5  Actual 7
[INFO] classifying image...
Predicted 4  Actual 2
[INFO] classifying image...
Predicted 6  Actual 8
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 2  Actual 2
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 4  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 5


Predicted 3  Actual 3
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 6  Actual 8
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 5  Actual 3
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 5  Actual 3
[INFO] classifying image...
Predicted 2  Actual 2
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...


Predicted 6  Actual 8
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 2  Actual 3
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 2  Actual 2
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...


Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 4  Actual 7
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 7  Actual 7
[INFO] classifying image...
Predicted 3  Actual 1
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 7  Actual 6
[INFO] classifying image...


Predicted 3  Actual 2
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 6  Actual 5
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 5  Actual 7
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 3  Actual 4
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 8  Actual 8
[INFO] classifying image...
Predicted 8  Actual 8
[INFO] classifying image...


...

Predicted 2  Actual 2
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 6  Actual 9
[INFO] classifying image...
Predicted 2  Actual 3
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 7  Actual 9
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 8  Actual 7
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 6  Actual 8
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 6
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 7  Actual 7
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 2  Actual 3
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 5  Actual 6
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 0  Actual 0
[INFO] classifying image...


Predicted 0  Actual 0
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 5  Actual 8
[INFO] classifying image...
Predicted 2  Actual 0
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 6  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 7
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...


Predicted 4  Actual 3
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 2  Actual 0
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 9  Actual 7
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 5  Actual 4
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 3  Actual 3
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 3
[INFO] classifying image...
Predicted 8  Actual 9
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...
Predicted 9  Actual 9
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...


Predicted 4  Actual 3
[INFO] classifying image...
Predicted 3  Actual 2
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 7  Actual 8
[INFO] classifying image...
Predicted 1  Actual 0
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 4  Actual 2
[INFO] classifying image...
Predicted 5  Actual 5
[INFO] classifying image...
Predicted 6  Actual 6
[INFO] classifying image...
Predicted 3  Actual 1
[INFO] classifying image...
Predicted 6  Actual 7
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 6  Actual 5
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 4  Actual 4
[INFO] classifying image...
Predicted 5  Actual 7
[INFO] classifying image...
Predicted 4  Actual 5
[INFO] classifying image...
Predicted 1  Actual 1
[INFO] classifying image...


In [6]:
custom_vgg_model.save("VGGFace50")